In [5]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
#import tensorflow_datasets as tfds
#import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (Dense,Flatten,SimpleRNN,InputLayer,Conv1D,LayerNormalization,Bidirectional,GRU,LSTM,BatchNormalization,Dropout,Input,MultiHeadAttention,Embedding,TextVectorization)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
#from google.colab import drive
#from google.colab import files
from tensorboard.plugins import projector

In [6]:
import pandas as pd
import tensorflow as tf

# Load the Excel sheet into a DataFrame
excel_file_path = r'C:\saipragatheeswar\thesis phase2\transformer\final dataset ai - human.xlsx'  # Replace with the path to your Excel file
df = pd.read_excel(excel_file_path)

def process_data(row):
    # Concatenate 'starttoken ' with the 'humanembedding' column and ' endtoken' to the 'AIembedding' column
    input_1 = row['Human']
    input_2 = 'starttoken ' + row['AI'] + ' endtoken'

    # Create tensors with the specified format
    tensor_1 = tf.constant([input_1], dtype=tf.string)
    tensor_2 = tf.constant([input_2], dtype=tf.string)

    return (tensor_1, tensor_2)

# Apply the process_data function to each row in the DataFrame and store the results in a list
data = [process_data(row) for _, row in df.iterrows()]



In [7]:
def data_generator():
    for item in data:
        yield item

# Create a dataset from the generator
dataset = tf.data.Dataset.from_generator(data_generator, output_signature=(
    tf.TensorSpec(shape=(1,), dtype=tf.string),
    tf.TensorSpec(shape=(1,), dtype=tf.string)
))

In [8]:
for element in dataset.take(3):
    print(element)

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b"I can't find my Glasses, can you help me?"], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'starttoken Give me a moment to locate your thing. endtoken'],
      dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b"I've lost my Belt, can you assist me?"], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"starttoken I'm searching for your object right now. endtoken"],
      dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b"I can't find my Necklace, can you help me?"], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"starttoken Sure, I'll help you find your object. endtoken"],
      dtype=object)>)


In [9]:
VOCAB_SIZE=2000
ENGLISH_SEQUENCE_LENGTH=32
FRENCH_SEQUENCE_LENGTH=32
EMBEDDING_DIM=256
BATCH_SIZE=128

english_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)
english_vectorize_layer1=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)

In [10]:
english_training_data = dataset.map(lambda x,y:x)
english_vectorize_layer.adapt(english_training_data)

In [11]:
english_training_data1=dataset.map(lambda x,y:y)
english_vectorize_layer1.adapt(english_training_data1)

In [12]:
import tensorflow as tf

# Assuming you already have your 'data' list defined as shown in the previous example
def process_data(row):
    # Concatenate 'starttoken ' with the 'humanembedding' column and ' endtoken' to the 'AIembedding' column
    input_1 = row['Human']
    input_2 = 'starttoken ' + row['AI']
    target = row['AI'] +' '+ 'endtoken'  # This will be the expected output

    return {'input_1': tf.constant([input_1]), 'input_2': tf.constant([input_2])}, tf.constant([target])

# Apply the process_data function to each row in the DataFrame and store the results in a list
data1 = [process_data(row) for _, row in df.iterrows()]
# Convert the list of data into a TensorFlow Dataset
dataset2 = tf.data.Dataset.from_generator(
    lambda: (item for item in data1),
    output_signature=(
        {
            'input_1': tf.TensorSpec(shape=(None,), dtype=tf.string),
            'input_2': tf.TensorSpec(shape=(None,), dtype=tf.string)
        },
        tf.TensorSpec(shape=(None,), dtype=tf.string)
    )
)
for i in dataset2.take(3):
  print(i)
# Define your vectorizer function


({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"I can't find my Glasses, can you help me?"], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Give me a moment to locate your thing.'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Give me a moment to locate your thing. endtoken'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"I've lost my Belt, can you assist me?"], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"starttoken I'm searching for your object right now."],
      dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"I'm searching for your object right now. endtoken"], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"I can't find my Necklace, can you help me?"], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"starttoken Sure, I'll help you find you

In [13]:
for i in english_vectorize_layer1.get_vocabulary():
  if i == 'endtoken':
    print(i)

endtoken


In [14]:
def vectorizer(inputs, output):
    return {
        'input_1': english_vectorize_layer(inputs['input_1']),
        'input_2': english_vectorize_layer1(inputs['input_2'])
    }, english_vectorize_layer1(output)

In [15]:
dataset11=dataset2.map(vectorizer)

In [16]:
for i in dataset11.take(3):
  print(i)

({'input_1': <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[ 8, 16, 15,  2, 60,  4,  3, 14,  5,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int64)>, 'input_2': <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[ 2, 26, 24, 12, 22,  7, 25,  4, 23,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int64)>}, <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[26, 24, 12, 22,  7, 25,  4, 23,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int64)>)
({'input_1': <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[18, 17,  2, 27,  4,  3, 12,  5,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int64)>, 'input_2': <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[ 2, 10, 29, 15,  4,  5, 17, 28,  0,  0, 

In [17]:
NUM_BATCHES=int(200000/BATCH_SIZE)
train_dataset=dataset11.take(int(0.9*NUM_BATCHES))
val_dataset=dataset11.skip(int(0.9*NUM_BATCHES))

In [18]:
for i in train_dataset.take(5):
  print(i)

({'input_1': <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[ 8, 16, 15,  2, 60,  4,  3, 14,  5,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int64)>, 'input_2': <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[ 2, 26, 24, 12, 22,  7, 25,  4, 23,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int64)>}, <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[26, 24, 12, 22,  7, 25,  4, 23,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int64)>)
({'input_1': <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[18, 17,  2, 27,  4,  3, 12,  5,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int64)>, 'input_2': <tf.Tensor: shape=(1, 32), dtype=int64, numpy=
array([[ 2, 10, 29, 15,  4,  5, 17, 28,  0,  0, 

In [19]:
def positional_encoding(model_size,SEQUENCE_LENGTH):
  output=[]
  for pos in range(SEQUENCE_LENGTH):
    PE=np.zeros((model_size))
    for i in range(model_size):
      if i%2==0:
        PE[i]=np.sin(pos/(10000**(i/model_size)))
      else:
        PE[i]=np.cos(pos/(10000**((i-1)/model_size)))
    output.append(tf.expand_dims(PE,axis=0))
  out=tf.concat(output,axis=0)
  out=tf.expand_dims(out,axis=0)
  return tf.cast(out,dtype=tf.float32)

In [20]:
class Embeddings(Layer):
  def __init__(self, sequence_length, vocab_size, embed_dim,):
    super(Embeddings, self).__init__()
    self.token_embeddings=Embedding(
        input_dim=vocab_size, output_dim=embed_dim)
    self.sequence_length = sequence_length
    self.vocab_size = vocab_size
    self.embed_dim = embed_dim

  def call(self, inputs):
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions=positional_encoding(
        self.embed_dim,self.sequence_length)
    return embedded_tokens + embedded_positions

  def compute_mask(self, inputs, mask=None):
    return tf.math.not_equal(inputs, 0)

In [21]:
class CustomSelfAttention(Layer):
  def __init__(self,model_size):
    super(CustomSelfAttention,self).__init__()
    self.model_size=model_size
  def call(self,query,key,value,masking):
    ######## compute scores
    score=tf.matmul(query,key,transpose_b=True)
    ######## scaling
    score/=tf.math.sqrt(tf.cast(self.model_size,tf.float32))
    ######## masking
    masking=tf.cast(masking,dtype=tf.float32)
    score+=(1.-masking)*-1e10
    ######## attention_weights
    attention=tf.nn.softmax(score,axis=-1)*masking
    ######## output
    head=tf.matmul(attention,value)
    return head

In [22]:
class CustomMultiHeadAttention(Layer):
  def __init__(self,num_heads,key_dim):
    super(CustomMultiHeadAttention,self).__init__()

    self.num_heads=num_heads
    self.dense_q=[Dense(key_dim//num_heads) for _ in range(num_heads)]
    self.dense_k=[Dense(key_dim//num_heads) for _ in range(num_heads)]
    self.dense_v=[Dense(key_dim//num_heads) for _ in range(num_heads)]
    self.dense_o=Dense(key_dim)
    self.self_attention=CustomSelfAttention(key_dim)

  def call(self,query,key,value,attention_mask):
    heads=[]

    for i in range(self.num_heads):
      print("hello", self.dense_q[i](query).shape)
      head=self.self_attention(self.dense_q[i](query),self.dense_k[i](key),
                              self.dense_v[i](value),attention_mask)
      heads.append(head)
    heads=tf.concat(heads,axis=2)
    heads=self.dense_o(heads)
    return heads

In [23]:
class TransformerEncoder(Layer):
    def __init__(self, embed_dim, dense_dim, num_heads,):
        super(TransformerEncoder, self).__init__()
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = CustomMultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim,
        )
        self.dense_proj=tf.keras.Sequential(
            [Dense(dense_dim, activation="relu"),
             Dense(embed_dim),]
        )
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):

      if mask is not None:
        mask = tf.cast(
            mask[:,tf.newaxis, :], dtype="int32")
        T = tf.shape(mask)[2]
        padding_mask = tf.repeat(mask,T,axis=1)
      attention_output = self.attention(
          query=inputs, key=inputs,value=inputs,
          attention_mask=padding_mask
      )

      proj_input = self.layernorm_1(inputs + attention_output)
      proj_output = self.dense_proj(proj_input)
      return self.layernorm_2(proj_input + proj_output)

In [24]:
class TransformerDecoder(Layer):
  def __init__(self, embed_dim, latent_dim, num_heads,):
    super(TransformerDecoder, self).__init__()
    self.embed_dim = embed_dim
    self.latent_dim = latent_dim
    self.num_heads = num_heads
    self.attention_1=MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )
    self.attention_2=MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )
    self.dense_proj = tf.keras.Sequential(
        [Dense(latent_dim, activation="relu"),Dense(embed_dim),]
    )
    self.layernorm_1=LayerNormalization()
    self.layernorm_2=LayerNormalization()
    self.layernorm_3=LayerNormalization()
    self.supports_masking = True
  def call(self, inputs, encoder_outputs, enc_mask, mask=None):


    if mask is not None:
      causal_mask=tf.linalg.band_part(
        tf.ones([tf.shape(inputs)[0],
                 tf.shape(inputs)[1],
                 tf.shape(inputs)[1]],dtype=tf.int32),-1,0)
      mask = tf.cast(
          mask[:,tf.newaxis, :], dtype="int32")
      enc_mask = tf.cast(
          enc_mask[:,tf.newaxis, :], dtype="int32")

      T = tf.shape(mask)[2]
      padding_mask = tf.repeat(mask,T,axis=1)
      cross_attn_mask = tf.repeat(enc_mask,T,axis=1)
      combined_mask=tf.minimum(padding_mask,causal_mask)

    attention_output_1 = self.attention_1(
        query=inputs,key=inputs,value=inputs,
        attention_mask=combined_mask,

    )

    out_1 = self.layernorm_1(inputs + attention_output_1)

    attention_output_2= self.attention_2(
        query=out_1,key=encoder_outputs,value=encoder_outputs,
        attention_mask=cross_attn_mask,

    )
    out_2 = self.layernorm_2(out_1 + attention_output_2)

    proj_output = self.dense_proj(out_2)
    return self.layernorm_3(out_2 + proj_output)

In [25]:
EMBEDDING_DIM=512
D_FF=2048
NUM_HEADS=8
NUM_LAYERS=1
NUM_EPOCHS=10

In [26]:
encoder_inputs=Input(shape=(None,), dtype="int64", name="input_1")
emb = Embeddings(ENGLISH_SEQUENCE_LENGTH,VOCAB_SIZE,EMBEDDING_DIM)
x = emb(encoder_inputs)
enc_mask = emb.compute_mask(encoder_inputs)

for _ in range(NUM_LAYERS):
  x=TransformerEncoder(EMBEDDING_DIM,D_FF,NUM_HEADS)(x)
encoder_outputs=x

decoder_inputs=Input(shape=(None,), dtype="int64", name="input_2")

x = Embeddings(FRENCH_SEQUENCE_LENGTH,VOCAB_SIZE,EMBEDDING_DIM)(decoder_inputs)
for i in range(NUM_LAYERS):
  x=TransformerDecoder(EMBEDDING_DIM,D_FF,NUM_HEADS)(x, encoder_outputs,enc_mask)
x=tf.keras.layers.Dropout(0.5)(x)
decoder_outputs=Dense(VOCAB_SIZE, activation="softmax")(x)

transformer = tf.keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)
transformer.summary()

hello (None, 32, 64)
hello (None, 32, 64)
hello (None, 32, 64)
hello (None, 32, 64)
hello (None, 32, 64)
hello (None, 32, 64)
hello (None, 32, 64)
hello (None, 32, 64)
Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embeddings (Embeddings)        (None, 32, 512)      1024000     ['input_1[0][0]']                
                                                                                                  
 embeddings_1 (Embe

In [27]:
class BLEU(tf.keras.metrics.Metric):
    def __init__(self,name='bleu_score'):
        super(BLEU,self).__init__()
        self.bleu_score=0

    def update_state(self,y_true,y_pred,sample_weight=None):
      y_pred=tf.argmax(y_pred,-1)
      self.bleu_score=0
      for i,j in zip(y_pred,y_true):
        tf.autograph.experimental.set_loop_options()

        total_words=tf.math.count_nonzero(i)
        total_matches=0
        for word in i:
          if word==0:
            break
          for q in range(len(j)):
            if j[q]==0:
              break
            if word==j[q]:
              total_matches+=1
              j=tf.boolean_mask(j,[False if y==q else True for y in range(len(j))])
              break

        self.bleu_score+=total_matches/total_words

    def result(self):
        return self.bleu_score/BATCH_SIZE

In [28]:
class Scheduler(LearningRateSchedule):
  def __init__(self, d_model, warmup_steps):
    super(Scheduler, self).__init__()
    self.d_model = tf.cast(d_model, tf.float64)
    self.warmup_steps = tf.cast(warmup_steps, dtype=tf.float64)

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float64)
    return (self.d_model**(-0.5))*tf.math.minimum(step**(-0.5), step * (self.warmup_steps ** -1.5))

In [29]:
WARM_UP_STEPS = 4000
lr_scheduled = Scheduler(EMBEDDING_DIM, WARM_UP_STEPS)

In [30]:
transformer.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = Adam(lr_scheduled, beta_1=0.9, beta_2=0.98, epsilon=1e-9),)

In [32]:
history=transformer.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200
    )

Epoch 1/200
1405/1405 [==============================] - 118s 84ms/step - loss: 0.0812 - val_loss: 0.9232
Epoch 2/200
1405/1405 [==============================] - 118s 84ms/step - loss: 0.0721 - val_loss: 1.0153
Epoch 3/200
1405/1405 [==============================] - 124s 88ms/step - loss: 0.0571 - val_loss: 1.1238
Epoch 4/200
1405/1405 [==============================] - 117s 83ms/step - loss: 0.0557 - val_loss: 1.2419
Epoch 5/200
1405/1405 [==============================] - 119s 85ms/step - loss: 0.0542 - val_loss: 1.2225
Epoch 6/200
1405/1405 [==============================] - 124s 88ms/step - loss: 0.0536 - val_loss: 1.2309
Epoch 7/200
1405/1405 [==============================] - 120s 86ms/step - loss: 0.0523 - val_loss: 1.2755
Epoch 8/200
1405/1405 [==============================] - 125s 89ms/step - loss: 0.0530 - val_loss: 1.2669
Epoch 9/200
1405/1405 [==============================] - 123s 87ms/step - loss: 0.0515 - val_loss: 1.2813
Epoch 10/200
1405/1405 [======================

In [33]:
index_to_word={x:y for x, y in zip(range(len(english_vectorize_layer1.get_vocabulary())),
                                   english_vectorize_layer1.get_vocabulary())}

In [34]:
print(index_to_word)

{0: '', 1: '[UNK]', 2: 'starttoken', 3: 'endtoken', 4: 'your', 5: 'object', 6: 'ill', 7: 'to', 8: 'you', 9: 'find', 10: 'im', 11: 'assist', 12: 'a', 13: 'of', 14: 'in', 15: 'for', 16: 'do', 17: 'right', 18: 'help', 19: 'locating', 20: 'course', 21: 'my', 22: 'moment', 23: 'thing', 24: 'me', 25: 'locate', 26: 'give', 27: 'best', 28: 'now', 29: 'searching', 30: 'sure', 31: 'i', 32: 'and', 33: 'that', 34: 'have', 35: 'dont', 36: 'information', 37: 'sorry', 38: 'the', 39: 'here', 40: 'with', 41: 'involves', 42: 'can', 43: 'today', 44: 'how', 45: 'is', 46: 'whats', 47: 'like', 48: 'cant', 49: 'good', 50: 'from', 51: 'but', 52: 'moving', 53: 'what', 54: 'on', 55: 'becoming', 56: 'hi', 57: 'are', 58: 'preferences', 59: 'hello', 60: 'often', 61: 'more', 62: 'hey', 63: 'afternoon', 64: 'world', 65: 'it', 66: 'evening', 67: 'morning', 68: 'also', 69: 'this', 70: 'be', 71: 'new', 72: 'left', 73: 'top', 74: 'our', 75: 'life', 76: 'its', 77: 'bottom', 78: 'time', 79: 'many', 80: 'understanding', 81

In [35]:
for i in index_to_word:
  if index_to_word[i] == 'endtoken':
    print(i)

3


In [36]:
def translator(english_sentence):
  tokenized_english_sentence=english_vectorize_layer([english_sentence])
  shifted_target='starttoken'

  for i in range(FRENCH_SEQUENCE_LENGTH):
    tokenized_shifted_target=english_vectorize_layer1([shifted_target])
    output=transformer.predict([tokenized_english_sentence,tokenized_shifted_target])

    french_word_index=tf.argmax(output,axis=-1)[0][i].numpy()
    current_word=index_to_word[french_word_index]
    if current_word=='endtoken':
      break
    shifted_target+=' '+current_word
  return shifted_target[11:]

In [38]:
# Assuming 'transformer' is your trained TensorFlow model
transformer.save_weights('transformer_weights200.h5')



In [ ]:
from google.colab import files

files.download('transformer_weights200.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>